In [54]:
import pandas as pd
import polars as pl
import os
from functions.fix_parquet_with_header_data import fix_parquet_with_header_data
from functions.prepare_electric_filenames_from_household_sensors import prepare_electric_filenames_from_household_sensors
from functions.aggregate_data_daily import aggregate_data_daily
from functions.prepare_electric_filenames_from_appliance import prepare_electric_filenames_from_appliance
from pathlib import Path


### Przygotowanie df-a z lokalizacjami plików do agregowania

In [2]:
folder_path = "../../Data/Parquet/household_sensors/sensordata"
household_sensors_data_to_process = prepare_electric_filenames_from_household_sensors(folder_path)

In [3]:
folder_path = "../../Data/Parquet/room_and_appliance_sensors/sensordata"
appliance_data_to_process = prepare_electric_filenames_from_appliance(folder_path)

In [4]:
len(household_sensors_data_to_process["home_id"].unique())

254

In [17]:
appliance_data_to_process

,home_id,electric,appliance_name,filename
0,61,electric-appliance,kettle,home61_kitchen695_sensor1967_electric-applianc...
1,61,electric-appliance,dishwasher,home61_kitchen695_sensor1969_electric-applianc...
2,61,electric-appliance,microwave,home61_kitchen695_sensor1971_electric-applianc...
3,62,electric-appliance,fridgefreezer,home62_kitchen710_sensor1779_electric-applianc...
4,62,electric-appliance,kettle,home62_kitchen710_sensor1780_electric-applianc...
...,...,...,...,...
212,311,electric-appliance,vacuumcleaner,home311_kitchen2857_sensor31438_electric-appli...
213,328,electric-appliance,kettle,home328_kitchen2980_sensor31003_electric-appli...
214,328,electric-appliance,toaster,home328_kitchen2980_sensor31004_electric-appli...
215,328,electric-appliance,fridgefreezer,home328_kitchen2980_sensor31005_electric-appli...


### Poprawienie pierwszej kolumny

In [6]:
df_org = pl.read_parquet("../../Data/Parquet/household_sensors/sensordata/home47_hall654_sensor1216c1220_electric-mains_electric-combined.parquet")
df = fix_parquet_with_header_data(df_org)

In [7]:
daily_data = aggregate_data_daily(df)

In [8]:
daily_data

date,daily_sum
date,i64
2016-08-10,6785970
2016-08-11,10097498
2016-08-12,8450189
2016-08-13,9321356
2016-08-14,16723329
…,…
2017-05-04,12720364
2017-05-05,10235043
2017-05-06,12310177


In [9]:
df = pl.read_parquet("../../Data/Parquet/room_and_appliance_sensors/sensordata/home61_kitchen695_sensor1967_electric-appliance_kettle.parquet")

In [10]:
df_clean = fix_parquet_with_header_data(df)

In [11]:
daily_kettle = aggregate_data_daily(df_clean)

In [13]:
daily_kettle

date,daily_sum
date,i64
2016-11-17,11763
2016-11-18,14256
2016-11-19,22827
2016-11-20,25511
2016-11-21,19826
…,…
2017-08-03,11549
2017-08-04,0
2017-08-05,0


In [18]:
df_home_id=pl.read_csv('../../Data/NAV/Features_HomeID.csv')
df_home_id

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [33]:
appliance_data_to_process_pl = pl.DataFrame(appliance_data_to_process)
appliance_data_to_process_pl

home_id,electric,appliance_name,filename
i64,str,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…"
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…"
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…"
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…"
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…"
…,…,…,…
311,"""electric-appliance""","""vacuumcleaner""","""home311_kitchen2857_sensor3143…"
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…"
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…"


In [40]:
df_joined = appliance_data_to_process_pl.join(
    df_home_id.unique(subset=["homeid_1"]),
    left_on='home_id',
    right_on='homeid_1',
    how='left'
)
df_joined

home_id,electric,appliance_name,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
311,"""electric-appliance""","""vacuumcleaner""","""home311_kitchen2857_sensor3143…",1,1,2,0,2,0,null,null,8,"""Don't own""","""flat"""
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [71]:
pl.DataFrame.with_columns

<function polars.dataframe.frame.DataFrame.with_columns(self, *exprs: 'IntoExpr | Iterable[IntoExpr]', **named_exprs: 'IntoExpr') -> 'DataFrame'>

In [ ]:
import polars as pl
from pathlib import Path

df_all = []

for i in df_joined['home_id'].unique().to_list():
    df_id = df_joined.filter(pl.col('home_id') == i)
    for k in df_id['filename']:
        df_id_file = df_id.filter(pl.col('filename') == k)
        device = df_id_file['appliance_name'][0]
        path_file = Path(f"../../Data/Parquet/room_and_appliance_sensors/sensordata/{k}")
        print(path_file)
        df = pl.read_parquet(path_file)
        df_clean = fix_parquet_with_header_data(df)
        daily_data = aggregate_data_daily(df_clean)  

 
        df_tmp = daily_data.with_columns([
            pl.lit(i).alias("home_id"),
            pl.lit(device).alias("device")
        ])
        df_all.append(df_tmp)

df_all = pl.concat(df_all)


..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home61_kitchen695_sensor1967_electric-appliance_kettle.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home61_kitchen695_sensor1969_electric-appliance_dishwasher.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home61_kitchen695_sensor1971_electric-appliance_microwave.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home62_kitchen710_sensor1779_electric-appliance_fridgefreezer.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home62_kitchen710_sensor1780_electric-appliance_kettle.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home62_kitchen710_sensor1781_electric-appliance_washingmachine.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home62_kitchen710_sensor1782_electric-appliance_dishwasher.parquet
..\..\Data\Parquet\room_and_appliance_sensors\sensordata\home62_kitchen710_sensor2400_electric-appliance_vacuumcleaner.parquet
..\..\D

In [78]:
df_joined.filter(pl.col('appliance_name') == 'kettle')

home_id,electric,appliance_name,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
63,"""electric-appliance""","""kettle""","""home63_kitchen720_sensor3760_e…",1,1,2,0,2,2,null,null,9,"""Don't own""","""house_or_bungalow"""
65,"""electric-appliance""","""kettle""","""home65_kitchen733_sensor3109_e…",1,1,2,0,2,1,27.0,15.0,11,"""Don't own""","""house_or_bungalow"""
73,"""electric-appliance""","""kettle""","""home73_kitchenliving781_sensor…",1,2,3,2,1,0,null,null,8,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
264,"""electric-appliance""","""kettle""","""home264_kitchen2437_sensor2076…",3,1,4,2,2,1,25.0,12.53,11,"""Own and use""","""house_or_bungalow"""
268,"""electric-appliance""","""kettle""","""home268_kitchen2471_sensor2153…",1,0,1,0,1,1,27.0,15.0,10,"""Don't own""","""house_or_bungalow"""
276,"""electric-appliance""","""kettle""","""home276_kitchen2561_sensor2207…",1,1,2,0,1,2,26.0,3.75,10,"""Own and use""","""flat"""


In [79]:
daily_kettle

date,daily_sum
date,i64
2016-11-17,11763
2016-11-18,14256
2016-11-19,22827
2016-11-20,25511
2016-11-21,19826
…,…
2017-08-03,11549
2017-08-04,0
2017-08-05,0


In [85]:
df_joined.filter(pl.col('home_id') == 328)['filename'][-1]
path_file = Path(f"../../Data/Parquet/room_and_appliance_sensors/sensordata/{df_joined.filter(pl.col('home_id') == 328)['filename'][-1]}")
path_file
clean_microwave = fix_parquet_with_header_data(pl.read_parquet(path_file))
daily_microwave = aggregate_data_daily(clean_microwave)
daily_microwave

date,daily_sum
date,i64
2018-05-26,3776
2018-05-27,0
2018-05-28,2662
2018-05-29,2392
2018-05-30,0


In [76]:
df_all

date,daily_sum,home_id,device
date,i64,i32,str
2016-11-17,11763,61,"""kettle"""
2016-11-18,14256,61,"""kettle"""
2016-11-19,22827,61,"""kettle"""
2016-11-20,25511,61,"""kettle"""
2016-11-21,19826,61,"""kettle"""
…,…,…,…
2018-05-26,3776,328,"""microwave"""
2018-05-27,0,328,"""microwave"""
2018-05-28,2662,328,"""microwave"""


Wygląda na ok :)

In [86]:
df_joined

home_id,electric,appliance_name,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,"""electric-appliance""","""kettle""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""dishwasher""","""home61_kitchen695_sensor1969_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,"""electric-appliance""","""microwave""","""home61_kitchen695_sensor1971_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,"""electric-appliance""","""fridgefreezer""","""home62_kitchen710_sensor1779_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,"""electric-appliance""","""kettle""","""home62_kitchen710_sensor1780_e…",1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
311,"""electric-appliance""","""vacuumcleaner""","""home311_kitchen2857_sensor3143…",1,1,2,0,2,0,null,null,8,"""Don't own""","""flat"""
328,"""electric-appliance""","""kettle""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,"""electric-appliance""","""toaster""","""home328_kitchen2980_sensor3100…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [89]:

df_all = df_all.with_columns([
    pl.col("device").str.strip_chars().str.to_lowercase()
])

df_joined = df_joined.with_columns([
    pl.col("appliance_name").str.strip_chars().str.to_lowercase()
])


df_panel_long = df_all.join(
    df_joined,
    left_on=["home_id", "device"],
    right_on=["home_id", "appliance_name"],
    how="left"
)


In [95]:
df_panel_long

date,daily_sum,home_id,device,electric,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
date,i64,i32,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
2016-11-17,11763,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-18,14256,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-19,22827,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-20,25511,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-21,19826,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-05-26,3776,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-27,0,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-28,2662,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [91]:
df_panel_long.write_parquet("..\..\Data\ML_DF\data_frame_panel_long.parquet")

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\jansl\AppData\Local\Temp\ipykernel_14016\4218702844.py:1: SyntaxWarning: invalid escape sequence '\.'
  df_panel_long.write_parquet("..\..\Data\ML_DF\data_frame_panel_long.parquet")


In [107]:
pivot_df = df_panel_long.pivot(
    values="daily_sum",
    index=["date",'home_id'],
    columns="device",
    aggregate_function='first'  # or "sum" if you expect multiple entries per date/device
)

C:\Users\jansl\AppData\Local\Temp\ipykernel_14016\2203284142.py:1: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivot_df = df_panel_long.pivot(


In [119]:
df_panel_long

date,daily_sum,home_id,device,electric,filename,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
date,i64,i32,str,str,str,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
2016-11-17,11763,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-18,14256,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-19,22827,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-20,25511,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
2016-11-21,19826,61,"""kettle""","""electric-appliance""","""home61_kitchen695_sensor1967_e…",1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018-05-26,3776,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-27,0,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
2018-05-28,2662,328,"""microwave""","""electric-appliance""","""home328_kitchen2980_sensor3101…",0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [120]:
df_home_id

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
62,1,1,2,0,2,2,36.0,16.16,7,"""Don't own""","""flat"""
…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat"""


In [121]:
df_home_id.join(
    pivot_df,
    left_on='homeid_1',
    right_on='home_id',
    how='left')

homeid_1,num_males,num_females,num_residents,num_children,num_employed,num_with_higher_education,daily_standing_charge_pence,unit_charge_pence_per_kwh,n_rooms,smart_automation,hometype,date,kettle,dishwasher,microwave,fridgefreezer,washingmachine,vacuumcleaner,washingmachinetumbledrier,other,toaster,freezer,fridge,dehumidifier,electricheater,tumbledrier,grill,appliance,electricoven
i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,str,str,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-17,11763,2969,342,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-18,14256,14482,9147,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-19,22827,7705,4858,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-20,25511,20033,3333,null,null,null,null,null,null,null,null,null,null,null,null,null,null
61,1,1,2,0,2,0,0.33,0.12,9,"""Don't own""","""house_or_bungalow""",2016-11-21,19826,133,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat""",2018-05-26,8791,null,3776,20097,null,null,null,null,0,null,null,null,null,null,null,null,null
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat""",2018-05-27,12940,null,0,16200,null,null,null,null,0,null,null,null,null,null,null,null,null
328,0,1,1,0,0,1,null,null,6,"""Don't own""","""flat""",2018-05-28,2934,null,2662,18679,null,null,null,null,0,null,null,null,null,null,null,null,null
